<a href="https://colab.research.google.com/github/gmkmuralikarthik/Motion-classifier/blob/main/LSTM_motion_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install tf-nightly
import numpy as np
import pandas as pd
from numpy import mean
from numpy import std
from numpy import dstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import LSTM, TimeDistributed, Dense
from keras.utils import to_categorical, plot_model
from matplotlib import pyplot
from keras.utils.np_utils import to_categorical



In [2]:
path = '/content/drive/My Drive/Colab Notebooks/motion_classifier'
dfwalk = pd.read_csv(path + '/walking.csv', header=None)
dfrun = pd.read_csv(path + '/running.csv', header=None)
dfidle = pd.read_csv(path + '/idle.csv', header=None)
columns = ['x','y','z','class']
dfwalk.columns= columns
dfrun.columns= columns
dfidle.columns= columns

In [3]:
dfwalk.loc[dfwalk['class']=='walking','class'] = 1
dfrun.loc[dfrun['class']=='running','class'] = 2
dfidle.loc[dfidle['class']=='idle', 'class'] = 0

In [4]:
dfwalk.isnull().any()

x        False
y        False
z        False
class    False
dtype: bool

In [5]:
dfrun.isnull().any() 

x        False
y        False
z        False
class    False
dtype: bool

In [6]:
dfidle.isnull().any() 

x        False
y        False
z        False
class    False
dtype: bool

In [7]:
train = dfrun.loc[:dfrun.shape[0]*.7]
test = dfrun.loc[dfrun.shape[0]*.7:dfrun.shape[0]]
train = train.append(dfwalk.loc[:dfwalk.shape[0]*.7])
test = test.append(dfwalk.loc[dfwalk.shape[0]*.7:dfwalk.shape[0]])
train = train.append(dfidle.loc[:dfidle.shape[0]*.7])
test = test.append(dfidle.loc[dfidle.shape[0]*.7:dfidle.shape[0]])
# X = dfidle.loc[:]
# X = X.append(dfwalk)
# X = X.append(dfrun)

In [8]:
testY = test['class']
trainY = train['class']
trainX = train[['x','y','z']]
testX = test[['x','y','z']]

In [9]:
# X = X.to_numpy()
# Y = Y.to_numpy()
testY = testY.to_numpy(testY)
trainY = trainY.to_numpy(trainY)
trainX = trainX.to_numpy()
testX = testX.to_numpy()
testY = np.array(testY,dtype='int64')
trainY = np.array(trainY,dtype='int64')
# Y = np.array(Y,dtype='int64')

In [10]:
# X = X.reshape((int(X.shape[0]/12),12,3))
def process(X,Y):
  test = []
  train = []
  t = 20
  for i in range(t,X.shape[0], t):
    test.append(X[i-t:i, :])
    train.append(np.bincount(Y[i-t:i]).argmax())
    # print(max(np.bincount(Y[i-12:i])))
  X = np.array(test)
  Y = np.array(train,dtype='float64')
  return X, Y
# X.shape


In [11]:
testx, testy = process(testX, testY)
trainx, trainy = process(trainX, trainY)
del trainX
del trainY
del testX
del testY

In [12]:
testy = to_categorical(testy)
trainy = to_categorical(trainy)
# trainx = X[:int(X.shape[0]*.7),:,:]
# testx = X[int(X.shape[0]*.7):,:,:]
# Y = to_categorical(Y)
# trainy = Y[:int(Y.shape[0]*.7)]
# testy = Y[int(Y.shape[0]*.7):]

In [13]:
trainx.shape

(1172, 20, 3)

In [14]:
trainy.shape

(1172, 3)

In [15]:
testx.shape

(502, 20, 3)

In [16]:
testy.shape

(502, 3)

In [17]:
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [18]:
def evaluate_model(trainx, trainy, testx, testy):
	verbose, epochs, batch_size = 0, 5, 16
	n_timesteps, n_features, n_outputs = trainx.shape[1], trainx.shape[2], trainy.shape[1]
	model = Sequential()
	model.add(LSTM(16, input_shape=(n_timesteps,n_features)))
	model.add(Dropout(0.2))
	# model.add(Dense(16, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	model.fit(trainx, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	loss, accuracy = model.evaluate(testx, testy, batch_size=batch_size, verbose=1)
	return loss, accuracy, model

In [19]:
def run_experiment(repeats=10):
	# load data
	# trainX, trainy, testX, testy = load_dataset()
	# repeat experiment
	scores = list()
	for r in range(repeats):
		loss, score, model = evaluate_model(trainx, trainy, testx, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		print('>#%d: %.3f' % (r+1, loss))
		scores.append(score)
	
	summarize_results(scores)
	return model

In [20]:
model = run_experiment()

32/32 [==============================] - 1s 2ms/step - loss: 0.1708 - accuracy: 0.9582
>#1: 95.817
>#1: 0.171
32/32 [==============================] - 1s 2ms/step - loss: 0.1517 - accuracy: 0.9602
>#2: 96.016
>#2: 0.152
32/32 [==============================] - 0s 2ms/step - loss: 0.2177 - accuracy: 0.9422
>#3: 94.223
>#3: 0.218
32/32 [==============================] - 0s 2ms/step - loss: 0.1912 - accuracy: 0.9442
>#4: 94.422
>#4: 0.191
32/32 [==============================] - 0s 2ms/step - loss: 0.2108 - accuracy: 0.9382
>#5: 93.825
>#5: 0.211
32/32 [==============================] - 0s 2ms/step - loss: 0.1590 - accuracy: 0.9582
>#6: 95.817
>#6: 0.159
32/32 [==============================] - 0s 2ms/step - loss: 0.2021 - accuracy: 0.9442
>#7: 94.422
>#7: 0.202
32/32 [==============================] - 0s 2ms/step - loss: 0.1861 - accuracy: 0.9442
>#8: 94.422
>#8: 0.186
32/32 [==============================] - 1s 2ms/step - loss: 0.2084 - accuracy: 0.9382
>#9: 93.825
>#9: 0.208
32/32 [===

In [21]:
model.summary() 

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 16)                1280      
_________________________________________________________________
dropout_9 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 51        
Total params: 1,331
Trainable params: 1,331
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.save('/content/drive/My Drive/Colab Notebooks/LSTM/motion_classifier.h5', include_optimizer=False)

In [26]:
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('/content/drive/My Drive/Colab Notebooks/LSTM/MotionClass.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmps9zn43cg/assets


INFO:tensorflow:Assets written to: /tmp/tmps9zn43cg/assets


In [27]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path='/content/drive/My Drive/Colab Notebooks/LSTM/MotionClass.tflite')
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[0.40458262 0.19491105 0.40050632]]


THE END